In [ ]:
import sys
sys.path.append("../../")
import json
import numpy as np
from PIL import Image
import pandas as pd
from DMIN.utils import bytes_to_image
from datasets import load_dataset

In [ ]:
result_df = pd.read_parquet("./output_dir/test_dataset.parquet")
train_df = load_dataset("huaweilin/DMin_mixed_datasets_8846", split="train").to_pandas()

In [ ]:
for idx, data_dict in result_df.iterrows():
    print("-"*20, idx, "-"*20)
    print(data_dict["prompt"])
    bytes_to_image(data_dict["image"]).show()

In [ ]:
def concat_images(base_image, img_list):
    widths_0, heights = base_image.size
    widths_1, heights = zip(*(i.size for i in img_list))

    gap = 5
    large_gap = 120

    total_width = widths_0 + sum(widths_1) + gap*(len(img_list) - 1) + large_gap
    max_height = max(heights)
    new_img = Image.new('RGB', (total_width, max_height), color = (255,255,255))
    
    x_offset = 0
    new_img.paste(base_image, (x_offset, 0))
    x_offset += base_image.width + large_gap

    for img in img_list:
        new_img.paste(img, (x_offset, 0))
        x_offset += img.width + gap

    return new_img

for idx in range(len(result_df)):
    base_image = bytes_to_image(result_df.iloc[idx]["image"])
    results_list = [json.loads(x) for x in open(f"./output_dir/results_{idx}.jsonl").readlines()]
    infl_list = [x["compressed_influence_list"][0] for x in results_list]
    sorted_index = np.argsort(infl_list)[-20:][::-1]
    concat_images(base_image.resize((512, 512)), [bytes_to_image(train_df.iloc[x]["image"]["bytes"]).resize((512, 512)) for x in sorted_index]).show()